In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

from keras.preprocessing import image
from keras.applications import vgg16
import joblib




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory





# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
covid_pneumonia= Path("../input/covid-wwo-pneumonia-chest-xray/Data/train/covid_with_PNEUMONIA")
covid_non_pneumonia=Path("../input/covid-wwo-pneumonia-chest-xray/Data/train/covid_without_PNEUMONIA")
images=[]
labels=[]
print('adding on')

# 1 => Covid_pneumonia
# 0 => covid_non_pneumonia

for i in covid_pneumonia.glob("*.jpeg"):
    image_data= image.load_img(i,target_size=(700,700))
    i_array= image.img_to_array(image_data)
    images.append(i_array)
    labels.append(1)

for i in covid_non_pneumonia.glob("*.jpeg"):
    
    
    image_data= image.load_img(i,target_size=(700,700))
    i_array= image.img_to_array(image_data)
    images.append(i_array)
    labels.append(0)




x_train= np.array(images)
y_train= np.array(labels)


#call vgg16 model
x_train = vgg16.preprocess_input(x_train)
pretrained_model= vgg16.VGG16(weights="../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",input_shape=(700,700,3),include_top=False)

features_x= pretrained_model.predict(x_train)

print('pretraining is done............................')

adding on
pretraining is done............................


In [3]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten

model = Sequential()
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

#train the model

model.fit(features_x,y_train,epochs=10,shuffle=True)

#predict the value
#image_data= image.load_img('../input/covid-wwo-pneumonia-chest-xray/Data/test/covid_with_PNEUMONIA/cwp(1).jpeg',target_size=(700,700))
#i_array= image.img_to_array(image_data)
#x_test= np.expand_dims(i_array,axis=0)
#print('the predicted value is ',model.predict(x_test))


model_structure = model.to_json()
f = Path("./pretrained_model_structure.json")
f.write_text(model_structure)

model.save_weights("./pretrained_model_weights.h5")

print('model is saved successfully')


img= image.load_img('../input/covid-wwo-pneumonia-chest-xray/Data/test/covid_with_PNEUMONIA/cwp(1).jpeg',target_size=(700,700))
img_arr= image.img_to_array(img)
img_arr= vgg16.preprocess_input(img_arr)
features_extracted_model= vgg16.VGG16(weights="imagenet",input_shape=(700,700,3),include_top=False)
img_arr= np.expand_dims(img_arr,axis=0)
features=features_extracted_model.predict(img_arr)
z = {0:'covid_non_pneumonia',1:'Covid_pneumonia'}
print('The predicted analysis for image is ',z[int(model.predict(features)[0])])





Epoch 1/10
12/12 [==============================] - 1s 24ms/step - loss: 60.0360 - accuracy: 0.7051
Epoch 2/10
12/12 [==============================] - 0s 26ms/step - loss: 12.4532 - accuracy: 0.8938
Epoch 3/10
12/12 [==============================] - 0s 28ms/step - loss: 3.6576 - accuracy: 0.9491
Epoch 4/10
12/12 [==============================] - 0s 25ms/step - loss: 1.6055 - accuracy: 0.9595
Epoch 5/10
12/12 [==============================] - 0s 20ms/step - loss: 0.5389 - accuracy: 0.9841
Epoch 6/10
12/12 [==============================] - 0s 20ms/step - loss: 0.1492 - accuracy: 0.9823
Epoch 7/10
12/12 [==============================] - 0s 20ms/step - loss: 0.5717 - accuracy: 0.9803
Epoch 8/10
12/12 [==============================] - 0s 20ms/step - loss: 0.0758 - accuracy: 0.9929
Epoch 9/10
12/12 [==============================] - 0s 20ms/step - loss: 0.1401 - accuracy: 0.9879
Epoch 10/10
12/12 [==============================] - 0s 19ms/step - loss: 0.0218 - accuracy: 0.9983
model i

In [4]:
covid_pneumonia= Path("../input/covid-wwo-pneumonia-chest-xray/Data/test/covid_with_PNEUMONIA")
covid_non_pneumonia=Path("../input/covid-wwo-pneumonia-chest-xray/Data/test/covid_without_PNEUMONIA")
images=[]
labels=[]
print('checking test data...........')

# 0 => Covid_pneumonia
# 1 => covid_non_pneumonia

for i in covid_pneumonia.glob("*.jpeg"):
    image_data= image.load_img(i,target_size=(700,700))
    i_array= image.img_to_array(image_data)
    images.append(i_array)
    labels.append(1)

for i in covid_non_pneumonia.glob("*.jpeg"):
    
    
    image_data= image.load_img(i,target_size=(700,700))
    i_array= image.img_to_array(image_data)
    images.append(i_array)
    labels.append(0)

x_test= np.array(images)
y_test= np.array(labels)
#call vgg16 model
x_test = vgg16.preprocess_input(x_test)
pretrained_model= vgg16.VGG16(weights="../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",input_shape=(700,700,3),include_top=False)
features_x= pretrained_model.predict(x_test)

score = model.evaluate(features_x,y_test,verbose=0)

print(f'Test accuracy of model is {str(score[1])}')




    


checking test data...........
Test accuracy of model is 0.9577465057373047
